In [ ]:

import gdown

# File ID from Google Drive
file_id = '109M0r6BP3H8LS2fNNtKl5cVPorXDYz8M'
url = f'https://drive.google.com/uc?id={file_id}'

# Download the file
gdown.download(url, output='file.bin', quiet=False)


Downloading...
From (original): https://drive.google.com/uc?id=109M0r6BP3H8LS2fNNtKl5cVPorXDYz8M
From (redirected): https://drive.google.com/uc?id=109M0r6BP3H8LS2fNNtKl5cVPorXDYz8M&confirm=t&uuid=ca2fc368-2f67-4aaf-a8d8-2e5c5b6cd90e
To: /content/file.bin
100%|██████████| 169M/169M [00:01<00:00, 99.3MB/s]


'file.bin'

In [1]:
import pickle
import json

# Step 1: Load .bin file as a pickle
with open("file.bin", "rb") as f:
    data = pickle.load(f)

# Step 2: Save as JSON
with open("output.json", "w", encoding="utf-8") as f:
    json.dump(data, f, indent=2)

print("✅ Successfully converted pickle .bin to output.json")


✅ Successfully converted pickle .bin to output.json


In [2]:
# Build lookup dict for O(1) access
address_lookup = {}

for obj in data:
    address = obj['address']
    address_lookup[address] = obj  # stores the full object


In [9]:
#address_lookup

address_lookup['34xp4vRoCGJym3xA7yCVPFHoCNxv4Twseo']

KeyError: '34xp4vRoCGJym3xA7yCVPFHoCNxv4Twseo'

In [ ]:
legacy = []
p2sh = []
bech32 = []
taproot = []
new_data = {}

for obj in data:
    address = obj['address']
    new_data[address] = obj  # You could also use [obj] if needed
    if address.startswith('1'):
        legacy.append(obj)
    elif address.startswith('3'):
        p2sh.append(obj)
    elif address.startswith('bc1p'):
        taproot.append(obj)
    elif address.startswith('bc1q'):
        bech32.append(obj)

# Result
print("Legacy:", len(legacy))
print("P2SH:", len(p2sh))
print("Bech32:", len(bech32))
print("Taproot:", len(taproot))


Legacy: 673198
P2SH: 589021
Bech32: 1471039
Taproot: 26506


In [ ]:
import itertools

def get_combinations(seed_lst):

  lst = seed_lst  # assuming it contains 2048 BIP39 words

  result = []
  count = 0

  # This will generate all possible 12-word phrases — which is 2048^12 combinations (very large!)
  # So you must limit it using a counter or you'll run out of memory and CPU.
  for p in itertools.product(lst, repeat=12):
      count += 1

      if count >= 10000000 and count <=20000000:  # limit to first 1000 phrases
        joined = ' '.join(p)
        result.append(joined)
  return result

result = get_combinations(english_lst)



ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-36-a4d7953be9d9>", line 20, in <cell line: 0>
    result = get_combinations(english_lst)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "<ipython-input-36-a4d7953be9d9>", line None, in get_combinations
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_traceback_()
          ^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'KeyboardInterrupt' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/ultratb.py", line 1101, 

In [26]:
ls = globals()

legacy[0]['1FeexV6bAHb8ybZjqQMjJrcCrHGW9sb6uF']


TypeError: string indices must be integers, not 'str'

In [35]:
len(result)

9999999

In [ ]:
!pip install bip-utils
!pip install mnemonic bip_utils


In [30]:
2048**12
# 5444517870735015415413993718908291383296 =  2048^12



5444517870735015415413993718908291383296

In [16]:
from mnemonic import Mnemonic
from bip_utils import (
    Bip39SeedGenerator,
    Bip44, Bip44Coins,
    Bip49, Bip49Coins,
    Bip84, Bip84Coins,
    Bip86, Bip86Coins,
    Bip44Changes
)
import requests
from bs4 import BeautifulSoup
import re
from concurrent.futures import ThreadPoolExecutor, as_completed



headers = {
    "User-Agent": "Mozilla/5.0"
}

def get_address_info(item):
    address = item["address"]
    url = f"https://blockchair.com/bitcoin/address/{address}"
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code != 200:
            return {"address": address, "tx_count": None, "balance_btc": None, "status": response.status_code}

        soup = BeautifulSoup(response.text, "html.parser")

        # Transaction count
        tab = soup.select_one("#tab-history-bitcoin-main")
        tx_count = None
        if tab:
            match = re.search(r'\(([\d,]+)\)', tab.text)
            if match:
                tx_count = int(match.group(1).replace(",", ""))

        # Balance
        balance = soup.select_one(".wb-ba")
        balance_btc = None
        if balance:
            balance_clean = balance.text.replace("+", "").replace(",", "").strip()
            try:
                balance_btc = float(balance_clean)
            except ValueError:
                pass

        return {"address": address, "tx_count": tx_count, "balance_btc": balance_btc, "status": 200}

    except Exception as e:
        return {"address": address, "tx_count": None, "balance_btc": None, "error": str(e), "status": "error"}

# # Number of concurrent threads
# MAX_WORKERS = 10

# results = []
# with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
#     futures = [executor.submit(get_address_info, item) for item in data]
#     for future in as_completed(futures):
#         result = future.result()
#         results.append(result)
#         print(result)

# # Final results
# print("\n✅ All Results:")
# for r in results:
#     print(r)

def generate_all_addresses_from_english_mnemonic(english_mnemonic: str):
    supported_languages = [
        "english", "french", "italian", "spanish", "chinese_simplified",
        "chinese_traditional", "japanese", "korean", "czech", "portuguese", "russian"
    ]

    mnemo_en = Mnemonic("english")
    indices = [mnemo_en.wordlist.index(w) for w in english_mnemonic.strip().split()]

    results = {}

    for lang in supported_languages:
        try:
            mnemo_lang = Mnemonic(lang)
            lang_mnemonic_words = [mnemo_lang.wordlist[i] for i in indices]
            lang_mnemonic = " ".join(lang_mnemonic_words)

            seed_bytes = Bip39SeedGenerator(lang_mnemonic).Generate()

            addr_legacy = Bip44.FromSeed(seed_bytes, Bip44Coins.BITCOIN)\
                .Purpose().Coin().Account(0).Change(Bip44Changes.CHAIN_EXT).AddressIndex(0).PublicKey().ToAddress()

            addr_p2sh = Bip49.FromSeed(seed_bytes, Bip49Coins.BITCOIN)\
                .Purpose().Coin().Account(0).Change(Bip44Changes.CHAIN_EXT).AddressIndex(0).PublicKey().ToAddress()

            addr_bech32 = Bip84.FromSeed(seed_bytes, Bip84Coins.BITCOIN)\
                .Purpose().Coin().Account(0).Change(Bip44Changes.CHAIN_EXT).AddressIndex(0).PublicKey().ToAddress()

            addr_taproot = Bip86.FromSeed(seed_bytes, Bip86Coins.BITCOIN)\
                .Purpose().Coin().Account(0).Change(Bip44Changes.CHAIN_EXT).AddressIndex(0).PublicKey().ToAddress()

            results[lang] = {
                "mnemonic": lang_mnemonic,
                "address_legacy_1": addr_legacy,
                "address_p2sh_3": addr_p2sh,
                "address_bech32_bc1": addr_bech32,
                "address_bech32m_bc1p": addr_taproot,
            }
        except Exception as e:
            results[lang] = {"error": str(e)}

    return results


def get_bal(english_seed):

  # Example usage:
  # english_seed = "abandon abandon abandon abandon abandon abandon abandon abandon abandon abandon abandon about"
  all_addresses = generate_all_addresses_from_english_mnemonic(english_seed)


  # Loop through all languages and addresses, call the checking function for each address:
  for lang, data in all_addresses.items():
      print(f"Language: {lang}")
      if "error" in data:
          print(f"  Error: {data['error']}")
      else:
          print(f"  Mnemonic: {data['mnemonic']}")

          addr_legacy_info = get_address_info({"address": data['address_legacy_1']})
          print(f"  Legacy (1): {data['address_legacy_1']} -> {addr_legacy_info}")

          addr_p2sh_info = get_address_info({"address": data['address_p2sh_3']})
          print(f"  P2SH (3): {data['address_p2sh_3']} -> {addr_p2sh_info}")

          addr_bech32_info = get_address_info({"address": data['address_bech32_bc1']})
          print(f"  Bech32 (bc1): {data['address_bech32_bc1']} -> {addr_bech32_info}")

          addr_bech32m_info = get_address_info({"address": data['address_bech32m_bc1p']})
          print(f"  Bech32m (bc1p): {data['address_bech32m_bc1p']} -> {addr_bech32m_info}")

      print("-" * 40)


Language: english
  Mnemonic: abandon abandon abandon abandon abandon abandon abandon abandon abandon abandon abandon about
  Legacy (1): 1LqBGSKuX5yYUonjxT5qGfpUsXKYYWeabA -> {'address': '1LqBGSKuX5yYUonjxT5qGfpUsXKYYWeabA', 'tx_count': 36, 'balance_btc': 0.0, 'status': 200}
  P2SH (3): 37VucYSaXLCAsxYyAPfbSi9eh4iEcbShgf -> {'address': '37VucYSaXLCAsxYyAPfbSi9eh4iEcbShgf', 'tx_count': 20, 'balance_btc': 0.0, 'status': 200}
  Bech32 (bc1): bc1qcr8te4kr609gcawutmrza0j4xv80jy8z306fyu -> {'address': 'bc1qcr8te4kr609gcawutmrza0j4xv80jy8z306fyu', 'tx_count': 150, 'balance_btc': 0.0, 'status': 200}
  Bech32m (bc1p): bc1p5cyxnuxmeuwuvkwfem96lqzszd02n6xdcjrs20cac6yqjjwudpxqkedrcr -> {'address': 'bc1p5cyxnuxmeuwuvkwfem96lqzszd02n6xdcjrs20cac6yqjjwudpxqkedrcr', 'tx_count': 20, 'balance_btc': 0.0, 'status': 200}
----------------------------------------
Language: french
  Mnemonic: abaisser abaisser abaisser abaisser abaisser abaisser abaisser abaisser abaisser abaisser abaisser abeille
  Legacy (

In [19]:
import os
from mnemonic import Mnemonic

def export_bip39_wordlists():
    supported_languages = [
        "english", "french", "italian", "spanish", "chinese_simplified",
        "chinese_traditional", "japanese", "korean", "czech", "portuguese", "russian"
    ]

    os.makedirs("output_lang_files", exist_ok=True)

    wordlists_by_lang = {}

    for lang in supported_languages:
        try:
            mnemo = Mnemonic(lang)
            wordlist = mnemo.wordlist

            # Save to file
            with open(f"output_lang_files/{lang}.txt", "w", encoding="utf-8") as f:
                for word in wordlist:
                    f.write(word + "\n")

            # Save to Python variable
            var_name = f"{lang.replace('-', '_')}_lst"
            globals()[var_name] = wordlist
            wordlists_by_lang[lang] = wordlist

            print(f"Saved: {lang}.txt and variable: {var_name}")

        except Exception as e:
            print(f"Error processing language {lang}: {e}")

    return wordlists_by_lang

# Run and get dictionary of wordlists
lang_wordlists = export_bip39_wordlists()

# Example usage:
# print(english_lst[:10])  # First 10 English words
# print(lang_wordlists["french"][:10])  # First 10 French words


Saved: english.txt and variable: english_lst
Saved: french.txt and variable: french_lst
Saved: italian.txt and variable: italian_lst
Saved: spanish.txt and variable: spanish_lst
Saved: chinese_simplified.txt and variable: chinese_simplified_lst
Saved: chinese_traditional.txt and variable: chinese_traditional_lst
Saved: japanese.txt and variable: japanese_lst
Saved: korean.txt and variable: korean_lst
Saved: czech.txt and variable: czech_lst
Saved: portuguese.txt and variable: portuguese_lst
Saved: russian.txt and variable: russian_lst
